In [1]:
import numpy as np
import tensorflow as tf

In [12]:
bandits = [0.2, 0, -0.5, -1]
num_bandits = len(bandits)

def pullBandit(bandit):
    result = np.random.randn(1)
    if result > bandit:
        return 1
    else:
        return -1
# the 4th bandit is the best

Agent

In [13]:
tf.reset_default_graph()

tf.set_random_seed(1)
policy = tf.Variable(tf.ones([num_bandits]))
chosen_action = tf.argmax(policy, 0)

reward_holder = tf.placeholder(shape=[1], dtype=tf.float32)
action_holder = tf.placeholder(shape=[1], dtype=tf.int32)
# it looks like tf.slice is equivalent to [] slice operator
# responsible_policy = tf.slice(policy, action_holder, [1])
responsible_policy = policy[action_holder[0]]
loss = tf.reduce_sum(-(tf.log(responsible_policy)*reward_holder))
train = tf.train.GradientDescentOptimizer(0.001).minimize(loss)

# Training
num_episodes = 1000
total_reward = np.zeros(num_bandits)
e = 0.1
np.random.seed(1)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for episode in range(num_episodes):
        
        if np.random.rand(1) < e:
            action = np.random.randint(num_bandits)
        else:
            action = sess.run(chosen_action)
        
        reward = pullBandit(bandits[action])
        _, ww, l = sess.run([train, policy, loss], 
                            feed_dict={reward_holder:[reward], action_holder:[action]})
        
        total_reward[action] += reward
        if episode % 50 == 0:
            print("Running reward for the {} bandits: {}".format(num_bandits, total_reward))
            print("loss: {}".format(l))
            print("policy: {}".format(ww))
print()
print("The agent think bandit {} is the best".format(np.argmax(ww)+1))
if np.argmax(ww) == np.argmax(-np.array(bandits)):
    print("The agent is right!")
else:
    print("The agent is wrong.....")

Running reward for the 4 bandits: [-1.  0.  0.  0.]
loss: 0.0
policy: [ 0.99900001  1.          1.          1.        ]
Running reward for the 4 bandits: [-2.  9.  0.  2.]
loss: 0.009923764504492283
policy: [ 0.99799901  1.00898302  1.          1.00199902]
Running reward for the 4 bandits: [-2.  5.  1.  3.]
loss: -0.004029489122331142
policy: [ 0.99799901  1.00503361  1.001001    1.00299704]
Running reward for the 4 bandits: [ -3.  13.   0.   3.]
loss: -0.01192594412714243
policy: [ 0.996997    1.01298547  1.00000203  1.00299704]
Running reward for the 4 bandits: [ -3.  15.   2.   3.]
loss: -0.013897789642214775
policy: [ 0.996997    1.01498103  1.00200105  1.00299704]
Running reward for the 4 bandits: [ -3.  13.   2.   3.]
loss: -0.011971530504524708
policy: [ 0.996997    1.0130316   1.002002    1.00299704]
Running reward for the 4 bandits: [ -3.  23.   2.   5.]
loss: -0.02166229672729969
policy: [ 0.996997    1.02287722  1.00200295  1.0049901 ]
Running reward for the 4 bandits: [ -6.

In [4]:
# repeat above chunk for multiple time, you will find out that sometime the agent will stuck in bad policy